In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.utils import class_weight

import numpy as np

C:\Users\Hsu-Hui\.conda\envs\GIS\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hsu-Hui\.conda\envs\GIS\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hsu-Hui\.conda\envs\GIS\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hsu-Hui\.conda\envs\GIS\lib\site-packages\tensorflow\python\framework\dty

In [2]:
tf.keras.backend.set_floatx('float32')

drop = 0.5
img_size = (320, 240)


In [3]:
model = Sequential([
    Conv2D(8, 5, activation = 'relu', input_shape = (img_size[0], img_size[1], 1)),
    MaxPool2D(5),
    Conv2D(16, 4, activation = 'relu'),
    MaxPool2D(2),
    Conv2D(32, 3, activation = 'relu'),
    Flatten(),
    Dense(32, activation = 'relu'),
    Dropout(drop),
    Dense(8, activation = 'relu'),
    Dense(3, activation = 'softmax')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [5]:
datagen = ImageDataGenerator(
    rescale = 1. / 255.,
    shear_range = 0.2,
    zoom_range = 0.05,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.05,
    brightness_range = [1, 1.5],
    horizontal_flip = True,
    dtype = tf.float32)

In [6]:
train_generator = datagen.flow_from_directory(
    'D:/squat_demo/new_squat_data',
    target_size = (320, 240),
    color_mode = 'grayscale',
    batch_size = 32,
    shuffle = True,
    class_mode='categorical')

Found 960 images belonging to 3 classes.


In [7]:
test_datagen = ImageDataGenerator(
    rescale = 1. / 255.,
    dtype = tf.float32)

test_generator = test_datagen.flow_from_directory(
    'D:/squat_demo/new_squat_data',
    target_size = img_size,
    color_mode = 'grayscale',
    batch_size = 16,
    shuffle = True,
    class_mode='categorical')


Found 960 images belonging to 3 classes.


In [8]:
class_weights = class_weight.compute_class_weight(
                   'balanced',
                   np.unique(train_generator.classes), 
                   train_generator.classes)

C:\Users\Hsu-Hui\.conda\envs\GIS\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2], y=[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [9]:
model.fit(train_generator, 
          epochs = 100,
          shuffle = True,
          validation_data = test_generator,
          class_weight = class_weights,
          workers = 8,
          max_queue_size = 512)

Epoch 1/100
30/30 [==============================] - 12s 408ms/step - loss: 1.1697 - acc: 0.3198 - val_loss: 1.0987 - val_acc: 0.3500
Epoch 2/100
30/30 [==============================] - 11s 350ms/step - loss: 1.1017 - acc: 0.3260 - val_loss: 1.0992 - val_acc: 0.3458
Epoch 3/100
30/30 [==============================] - 11s 350ms/step - loss: 1.0988 - acc: 0.3531 - val_loss: 1.0979 - val_acc: 0.3458
Epoch 4/100
30/30 [==============================] - 11s 357ms/step - loss: 1.0970 - acc: 0.3667 - val_loss: 1.0982 - val_acc: 0.3500
Epoch 5/100
30/30 [==============================] - 11s 352ms/step - loss: 1.0979 - acc: 0.3323 - val_loss: 1.0967 - val_acc: 0.5115
Epoch 6/100
30/30 [==============================] - 10s 346ms/step - loss: 1.0895 - acc: 0.3646 - val_loss: 1.0846 - val_acc: 0.3458
Epoch 7/100
30/30 [==============================] - 11s 353ms/step - loss: 1.0884 - acc: 0.3938 - val_loss: 1.0821 - val_acc: 0.4646
Epoch 8/100
30/30 [==============================] - 11s 352ms

Epoch 62/100
30/30 [==============================] - 10s 343ms/step - loss: 0.5934 - acc: 0.7333 - val_loss: 0.4545 - val_acc: 0.8490
Epoch 63/100
30/30 [==============================] - 10s 345ms/step - loss: 0.5657 - acc: 0.7458 - val_loss: 0.4571 - val_acc: 0.8344
Epoch 64/100
30/30 [==============================] - 10s 346ms/step - loss: 0.5965 - acc: 0.7573 - val_loss: 0.4469 - val_acc: 0.8687
Epoch 65/100
30/30 [==============================] - 10s 345ms/step - loss: 0.5406 - acc: 0.7646 - val_loss: 0.4630 - val_acc: 0.8250
Epoch 66/100
30/30 [==============================] - 10s 348ms/step - loss: 0.5814 - acc: 0.7479 - val_loss: 0.4464 - val_acc: 0.8469
Epoch 67/100
30/30 [==============================] - 10s 348ms/step - loss: 0.5641 - acc: 0.7708 - val_loss: 0.4374 - val_acc: 0.8479
Epoch 68/100
30/30 [==============================] - 10s 346ms/step - loss: 0.5319 - acc: 0.7573 - val_loss: 0.4327 - val_acc: 0.8490
Epoch 69/100
30/30 [==============================] - 1

In [25]:
model.save('new_squat_model.h5')

In [3]:
import cv2
import ffmpeg
from tensorflow.keras.models import load_model

def checkVideoRotation(videoPath):
    metadata = ffmpeg.probe(videoPath)

    code = None
    print(metadata)
    if int(metadata['streams'][0]['tags']['rotate']) == 90:
        code = cv2.ROTATE_90_CLOCKWISE
    elif int(metadata['streams'][0]['tags']['rotate']) == 180:
        code = cv2.ROTATE_180
    elif int(metadata['streams'][0]['tags']['rotate']) == 270:
        code = cv2.ROTATE_90_COUNTERCLOCKWISE

    return code

def frameForModel(frame):
    #print(frame)
    f = cv2.resize(frame, (320, 240))
    f = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY).astype(np.float32)
    f = np.expand_dims(f, axis = 0)
    f = np.expand_dims(f, axis = -1)
    f /= 255.
    
    return f
    

In [4]:
tf.enable_eager_execution() # 修正AttributeError: 'Tensor' object has no attribute 'numpy'

In [24]:
videoPath = 'D:/squat_demo/squat_test.mp4'
#videoPath = 'D:/Squats/v_BodyWeightSquats_g01_c01.avi'
modelPath = 'new_squat_model.h5'
model = load_model(modelPath)

font= cv2.FONT_HERSHEY_SIMPLEX
fontScale= 1
fontColor= (255, 0, 0)
lineType= 2


count = 0
currentState = [-1, 0]
minStrength = 3
canIncreaseCount = False

cap = cv2.VideoCapture(videoPath)
#cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
#cv2.resizeWindow('Frame', 540, 960)

#rotateCode = checkVideoRotation(videoPath)
rotateCode = None
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'XVID'), 30, size,1)


#out = cv2.VideoWriter('out1.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, size,1)


In [25]:
if not cap.isOpened():
    print("Error while opening the video:", videoPath)
    
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        if rotateCode is not None:
            frame = cv2.rotate(frame, rotateCode)
            
        pred = model(frameForModel(frame)).numpy()
        print(pred)
        c = np.argmax(pred)
        #print(c)
        category = 'None'
        state = -1
        
        if pred[0][c] >= 0.5:
            if c == 0:
                category = "Lower"
                state = 0
            elif c == 1:
                category = "Middle"
                state = 1
            elif c == 2:
                category = "Upper"
                state = 2
            
        if currentState[0] == state:
            currentState[1] += 1
        else:
            currentState = [state, 1]
            
        if currentState[1] > minStrength:
            if currentState[0] == 1:
                canIncreaseCount = True
            elif currentState[0] == 2 and canIncreaseCount:
                count += 1
                canIncreaseCount = False
        
        cv2.putText(frame,
                    category, 
                    (10, 80), 
                    font, 
                    fontScale,
                    fontColor,
                    lineType)
        
        cv2.putText(frame,
                    "Count: " + str(count), 
                    (10, 160), 
                    font, 
                    fontScale,
                    fontColor,
                    lineType)
        
        
        #v2.imshow('Frame', frame)
        out.write(frame)

        # if cv2.waitKey(25) & 0xFF == ord('q'):
        #     break
    else:
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()

[[0.01303054 0.33139476 0.6555747 ]]
[[0.01285259 0.3349628  0.6521846 ]]
[[0.01290693 0.34067735 0.6464157 ]]
[[0.01187665 0.3380306  0.6500927 ]]
[[0.0113913  0.33049336 0.6581153 ]]
[[0.0225472  0.35489666 0.62255615]]
[[0.04369516 0.39436164 0.5619432 ]]
[[0.06896171 0.5022342  0.42880404]]
[[0.10571315 0.61908436 0.2752025 ]]
[[0.16270266 0.67871034 0.15858705]]
[[0.18887405 0.6788055  0.13232046]]
[[0.21069731 0.66320765 0.12609504]]
[[0.35876077 0.5808215  0.06041773]]
[[0.4227542 0.5301375 0.0471083]]
[[0.4477563  0.5149616  0.03728219]]
[[0.40834364 0.55566114 0.03599519]]
[[0.41457418 0.5562713  0.02915451]]
[[0.5267076  0.46522593 0.00806651]]
[[0.6070984  0.3902618  0.00263979]]
[[0.66527987 0.33340314 0.00131698]]
[[7.7812916e-01 2.2163481e-01 2.3605407e-04]]
[[7.9522324e-01 2.0457353e-01 2.0328900e-04]]
[[8.5927749e-01 1.4065929e-01 6.3244501e-05]]
[[9.0480977e-01 9.5172949e-02 1.7252412e-05]]
[[8.9789778e-01 1.0207710e-01 2.5101777e-05]]
[[9.1345251e-01 8.6531460e-02 1.6

[[0.33193475 0.6121981  0.05586712]]
[[0.31767648 0.61179817 0.07052542]]
[[0.30223083 0.6075273  0.09024192]]
[[0.26066512 0.6115477  0.12778714]]
[[0.14801216 0.5969269  0.25506097]]
[[0.06494092 0.4898067  0.4452524 ]]
[[0.02815093 0.40808478 0.56376433]]
[[0.02900616 0.41336504 0.5576288 ]]
[[0.03136075 0.409038   0.5596013 ]]
[[0.03468994 0.40075573 0.56455433]]
[[0.03068492 0.36871132 0.6006038 ]]
[[0.03167463 0.39376423 0.5745611 ]]
[[0.02147551 0.38354346 0.5949811 ]]
[[0.0148925  0.36707956 0.6180279 ]]
[[0.01848853 0.38078955 0.600722  ]]
[[0.02076308 0.3843084  0.59492856]]
[[0.02594794 0.39501032 0.5790418 ]]
[[0.02751996 0.40095955 0.57152045]]
[[0.02551329 0.39191693 0.5825698 ]]
[[0.02274102 0.3808406  0.5964183 ]]
[[0.0234684  0.38165662 0.59487504]]
[[0.02231698 0.3789455  0.5987376 ]]
[[0.02302856 0.383498   0.5934735 ]]
[[0.01958251 0.37152055 0.6088969 ]]
[[0.03151202 0.39226496 0.576223  ]]
[[0.04839534 0.4192476  0.53235716]]
[[0.07262138 0.5123134  0.4150651 ]]
[

In [19]:
modelPath = 'new_squat_model.h5'
model = load_model(modelPath)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 316, 236, 8)       208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 105, 78, 8)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 102, 75, 16)       2064      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 51, 37, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 49, 35, 32)        4640      
_________________________________________________________________
flatten (Flatten)            (None, 54880)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1